In [6]:
from typing import Any
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import pytorch_lightning as pl
from torchmetrics import Accuracy, F1Score
from torchmetrics.aggregation import MeanMetric

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np

## I. Model

In [7]:
class LSTM1(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=104, batch_first=True)
        self.fc_layers = nn.Sequential(
            nn.Linear(in_features=104, out_features=52),
            nn.Linear(in_features=52, out_features=26),
            nn.Linear(in_features=26, out_features=1)
        )
        self.Tanh = nn.Tanh()

        
    def forward(self, x):
        x, (_, _) = self.lstm(x)
        x = self.Tanh(x)
        x = self.fc_layers(x)
        return x
    

class LSTM2(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=104, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=104, hidden_size=52, batch_first=True)
        self.fc_layers = nn.Linear(in_features=52, out_features=1)
        self.Tanh = nn.Tanh()

        
    def forward(self, x):
        x, (_, _) = self.lstm1(x)
        x = self.Tanh(x)
        x, (_, _) = self.lstm2(x)
        x = self.Tanh(x)
        x = self.fc_layers(x)
        return x
    


class NN(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss_fn = F.mse_loss


    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("validation_loss", loss)
                 
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("validation_loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("test_loss", loss)
        
        return loss

    def predict_step(self, batch, batch_idx):
        loss, scores , y = self._common_step(batch, batch_idx)
        preds = torch.argmax(scores, dim=1)
        return preds

    def _common_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0), -1)
        scores = self.model(x)
        loss = self.loss_fn(scores, y[:,0].reshape(1, 1))
        return loss, scores, y

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

## II. Dataset

In [8]:
from pathlib import Path
import pandas as pd
from torch.utils.data import Subset
from sklearn.preprocessing import MinMaxScaler

def data_construct(sale_data, time_step=3, sku_list= ["SKU 1st", "SKU 2nd", "SKU 3rd"]):

    len_data = len(sale_data[sku_list[0]]) - 3 
    data = np.empty([len(sku_list), len_data, time_step+1])

    for sku_index, sku_val in enumerate(sku_list):
        
        len_data = len(sale_data[sku_val]) - time_step 
        sku_data = np.array(sale_data[sku_val]).reshape(-1, 1)

        # Normalizing data
        normalizer = MinMaxScaler().fit(sku_data[:-52])
        sku_data = normalizer.transform(sku_data).flatten()

        # Merging data
        x = np.array([sku_data[i:i+time_step] for i in range(len_data)])
        y = sku_data[time_step:].reshape(len_data,1)

        # Check data dimensions
        if len(sku_list) > 1: 
            data[sku_index, :, :] = np.concatenate((x, y), axis=1)
        else:
            data = np.concatenate((x, y), axis=1)
    
    return data


class SaleDataset(Dataset):
    def __init__(self, excel_file, sku_list, time_step=3):
        """Initializes instance of class StudentsPerformanceDataset.
        Args:
            excel_file (str): Path to the csv file with the sale data.
        """

        sale_data = pd.read_excel(excel_file, 
                     sheet_name= sku_list,
                     usecols="D",
                     dtype=np.float32,
                     )
        
        self.data = torch.Tensor(data_construct(sale_data, time_step, sku_list)).clone().detach()
        if len(sku_list) > 1:
            self.data = torch.permute(self.data, (1, 2, 0))
        
            # Save target and predictors
            self.x = self.data[:, :time_step, :]
            self.y = self.data[:, -1, :]
        else:
            self.x = self.data[:, :time_step]
            self.y = self.data[:, -1]


    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return self.x[idx].clone().detach(), self.y[idx].clone().detach()
    
excel_file =r"C:\Users\Dave\Documents\GitHub\Projects\LSTM_TimeSeries\dataset\Sale_data.xlsm"
sku_list = ["SKU 1st"]
time_step = 3
a = SaleDataset(excel_file, sku_list, time_step)
a.y

tensor([0.1897, 0.0038, 0.0307, 0.2818, 0.1922, 0.2937, 0.0000, 0.0451, 0.2248,
        0.2868, 0.4458, 0.4796, 0.4746, 0.4546, 0.5272, 0.3262, 0.4590, 0.4847,
        0.5930, 0.6243, 0.4984, 0.4878, 0.4734, 0.1616, 0.0770, 0.2786, 0.0463,
        0.2292, 0.1703, 0.0182, 0.0776, 0.0407, 0.2154, 0.2655, 0.2561, 0.2279,
        0.4922, 0.4364, 0.5529, 0.5623, 0.3907, 0.4696, 0.4433, 0.3431, 0.5441,
        0.3181, 0.3338, 0.4101, 0.4208, 0.3319, 0.4033, 0.4847, 0.3776, 0.1916,
        0.2185, 0.4696, 0.3801, 0.4815, 0.1879, 0.2329, 0.4126, 0.4746, 0.6337,
        0.6675, 0.6625, 0.6425, 0.7151, 0.5141, 0.6468, 0.6725, 0.7808, 0.8121,
        0.6863, 0.6756, 0.6612, 0.3494, 0.2649, 0.4665, 0.2342, 0.4170, 0.3582,
        0.2060, 0.2655, 0.2286, 0.4033, 0.4534, 0.4440, 0.4158, 0.6800, 0.6243,
        0.7408, 0.7502, 0.5786, 0.6575, 0.6312, 0.5310, 0.7320, 0.5059, 0.5216,
        0.5980, 0.6086, 0.5197, 0.5911, 0.6725, 0.5654, 0.3795, 0.4064, 0.6575,
        0.5679, 0.6694, 0.3757, 0.4208, 

In [9]:
class SaleDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers, sku_list, time_step):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.sku_list = sku_list
        self.time_step = time_step

    # def prepare_data(self):
        

    def setup(self, stage):
        entire_dataset = SaleDataset(self.data_dir, self.sku_list, self.time_step)
        test_size = 52

        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            self.train_ds = Subset(entire_dataset, list(range(len(entire_dataset)-test_size)))
            self.train_ds, self.val_ds = random_split(self.train_ds, 
                                                    [int(len(self.train_ds)*0.85), len(self.train_ds) - int(len(self.train_ds)*0.85)])

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.test_ds =  Subset(entire_dataset, list(range(len(entire_dataset)-test_size, len(entire_dataset))))

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            # persistent_workers=True,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            # persistent_workers=True,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            # persistent_workers=True,
            shuffle=False,
        )
    
excel_file =r"C:\Users\Dave\Documents\GitHub\Projects\LSTM_TimeSeries\dataset\Sale_data.xlsm"
sku_list = ["SKU 1st", "SKU 2nd"]
dm = SaleDataModule(data_dir=excel_file, batch_size=1, num_workers=0, sku_list=sku_list, time_step=time_step)
dm.setup("fit")

dm.train_ds[0]

(tensor([[0.4064, 0.5602],
         [0.6575, 0.5074],
         [0.5679, 0.6805]]),
 tensor([0.6694, 0.5334]))

# III. Train model

In [10]:
pl.seed_everything(33)

# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_workers = 0

# Hyperparameters
learning_rate = 0.001
batch_size = 1
num_epochs = 10
time_step = 3 

# data
excel_file =r"C:\Users\Dave\Documents\GitHub\Projects\LSTM_TimeSeries\dataset\Sale_data.xlsm"
sku_list = ["SKU 1st", "SKU 2nd", "SKU 3rd"]
input_size = 3 if len(sku_list) == 1 else 9



if __name__ == "__main__":
    # Load Data
    dm = SaleDataModule(data_dir=excel_file, batch_size=batch_size, num_workers=num_workers, sku_list=sku_list, time_step=time_step)
    # Initialize network
    model = NN(LSTM2(input_size=input_size))

    trainer = pl.Trainer(accelerator="cpu", devices=1, min_epochs=1, max_epochs=num_epochs, log_every_n_steps=10)
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)


Seed set to 33
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


cpu



  | Name  | Type  | Params
--------------------------------
0 | model | LSTM2 | 80.8 K
--------------------------------
80.8 K    Trainable params
0         Non-trainable params
80.8 K    Total params
0.323     Total estimated model params size (MB)


C:\Users\Dave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\Dave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 130/130 [00:01<00:00, 81.94it/s, v_num=114]

...

Epoch 9: 100%|██████████| 130/130 [00:01<00:00, 81.29it/s, v_num=114]


C:\Users\Dave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Validation DataLoader 0: 100%|██████████| 23/23 [00:00<00:00, 217.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     validation_loss       0.014160525985062122
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


C:\Users\Dave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 52/52 [00:00<00:00, 291.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.015026570297777653
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
